In [22]:
# import libraries
%matplotlib tk
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
import numpy as np

imgpath = 'fronton1.png'
# imgpath = 'topdown1.png'
img = cv2.imread(imgpath)

image_points = np.loadtxt(f'{imgpath[:-4]}_image_points_2D.txt')
# Convert to float32
image_points = np.array(image_points, dtype=np.float32)

In [23]:
# World Points
# Since the image is a badminton court, we can define the coordinates as follows
LEFT_BOUNDARY_X = 0
LEFT_SIDELINE_X = 0.46
CENTRE_LINE_X = 0.46 + 2.59
RIGHT_SIDELINE_X = 0.46 + 2.59 + 2.59
RIGHT_BOUNDARY_X = 0.46 + 2.59 + 2.59 + 0.46
BOTTOM_BOUNDARY_Y = 0
BOTTOM_LONG_SERVICE_LINE  = 0.76
BOTTOM_SHORT_SERVICE_LINE = 0.76 + 3.96
NET_Y = 0.76 + 3.96 + 1.98
NET_HEIGHT = 1.55
TOP_SHORT_SERVICE_LINE = 0.76 + 3.96 + 3.96
TOP_LONG_SERVICE_LINE = 0.76 + 3.96 + 3.96 + 3.96
TOP_BOUNDARY_Y = 0.76 + 3.96 + 3.96 + 3.96 + 0.76

# Selected points gt
objp = np.zeros((1, 32, 3), np.float32)

objp[0, 0:5, 0] = [LEFT_BOUNDARY_X,LEFT_SIDELINE_X, CENTRE_LINE_X, RIGHT_SIDELINE_X, RIGHT_BOUNDARY_X]
objp[0, 0:5, 1] = np.ones(5) * BOTTOM_BOUNDARY_Y
objp[0, 0:5, 2] = [0, 0, 0, 0, 0]
objp[0, 5:10, 0] = [LEFT_BOUNDARY_X,LEFT_SIDELINE_X, CENTRE_LINE_X, RIGHT_SIDELINE_X, RIGHT_BOUNDARY_X]
objp[0, 5:10, 1] = np.ones(5) * BOTTOM_LONG_SERVICE_LINE
objp[0, 5:10, 2] = [0, 0, 0, 0, 0]
objp[0, 10:15, 0] = [LEFT_BOUNDARY_X,LEFT_SIDELINE_X, CENTRE_LINE_X, RIGHT_SIDELINE_X, RIGHT_BOUNDARY_X]
objp[0, 10:15, 1] = np.ones(5) * BOTTOM_SHORT_SERVICE_LINE
objp[0, 10:15, 2] = [0, 0, 0, 0, 0]
objp[0, 15:17, 0] = [LEFT_BOUNDARY_X, RIGHT_BOUNDARY_X]
objp[0, 15:17, 1] = np.ones(2) * NET_Y
objp[0, 15:17, 2] = [0, 0] 
objp[0, 17:22, 0] = [LEFT_BOUNDARY_X,LEFT_SIDELINE_X, CENTRE_LINE_X, RIGHT_SIDELINE_X, RIGHT_BOUNDARY_X]
objp[0, 17:22, 1] = np.ones(5) * TOP_SHORT_SERVICE_LINE
objp[0, 17:22, 2] = [0, 0, 0, 0, 0]
objp[0, 22:27, 0] = [LEFT_BOUNDARY_X,LEFT_SIDELINE_X, CENTRE_LINE_X, RIGHT_SIDELINE_X, RIGHT_BOUNDARY_X]
objp[0, 22:27, 1] = np.ones(5) * TOP_LONG_SERVICE_LINE
objp[0, 22:27, 2] = [0, 0, 0, 0, 0]
objp[0, 27:32, 0] = [LEFT_BOUNDARY_X,LEFT_SIDELINE_X, CENTRE_LINE_X, RIGHT_SIDELINE_X, RIGHT_BOUNDARY_X]
objp[0, 27:32, 1] = np.ones(5) * TOP_BOUNDARY_Y
objp[0, 27:32, 2] = [0, 0, 0, 0, 0]

world_points = objp[0]

# Save world points
np.savetxt('world_points_3D.txt', world_points)


In [24]:

world_points_temp = []
image_points_temp = []
world_points_temp.append(world_points)
image_points_temp.append(image_points)

ret, camera_matrix, distortion_coefficients, rotation_vectors, translation_vectors = cv2.calibrateCamera(world_points_temp, image_points_temp, (img.shape[1], img.shape[0]), None, None)

# Check Reprojection Error
mean_error = 0
for i in range(len(world_points_temp)):
    imgpoints2, _ = cv2.projectPoints(world_points_temp[i], rotation_vectors[i], translation_vectors[i], camera_matrix, distortion_coefficients)

# Visualize
img = cv2.imread(imgpath)
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(20,20),dpi=100)
plt.scatter(imgpoints2[:,0,0], imgpoints2[:,0,1], c='r', s=100)
plt.imshow(img)
plt.show()


In [25]:


distortion_params = distortion_coefficients.ravel()

print(f'camera_matrix: {camera_matrix}')
print(f'distortion_coefficients: {distortion_coefficients}')
print(f'rotation_vectors: {rotation_vectors}')

new_camera_matrix, roi = cv2.getOptimalNewCameraMatrix(camera_matrix, distortion_coefficients, (img.shape[1], img.shape[0]), 1, (img.shape[1], img.shape[0]))

print(f'new_camera_matrix: {new_camera_matrix}')
print(f'roi: {roi}')

undistorted_img = cv2.undistort(img, camera_matrix, distortion_coefficients, None, new_camera_matrix)

# crop the image
x, y, w, h = roi
# x-=75
# w+=115
# y-=15
# h+=30

undistorted_img_cropped = undistorted_img[y:y+h, x:x+w]
plt.figure(figsize=(20,20),dpi=100)
plt.imshow(undistorted_img_cropped)
plt.show()

# Save the 3x3 camera matrix

np.savetxt(f'{imgpath[:-4]}_camera_matrix.txt', camera_matrix)

# Save the 3x3 new camera matrix

np.savetxt(f'{imgpath[:-4]}_new_camera_matrix.txt', new_camera_matrix)

# Save the distortion coefficients

np.savetxt(f'{imgpath[:-4]}_distortion_coefficients.txt', distortion_coefficients)

camera_matrix: [[1.13918437e+03 0.00000000e+00 7.92821860e+02]
 [0.00000000e+00 1.18346621e+03 1.23328737e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
distortion_coefficients: [[-0.32281444  0.21812507 -0.05371257 -0.00114272 -0.05419283]]
rotation_vectors: (array([[ 2.26557185],
       [ 0.02385197],
       [-0.02439056]]),)
new_camera_matrix: [[782.92565918   0.         740.55012595]
 [  0.         876.03704834 908.622262  ]
 [  0.           0.           1.        ]]
roi: (196, 10, 1186, 1563)


In [26]:
# Get the undistorted image points

undistorted_image_points = cv2.undistortPoints(image_points, camera_matrix, distortion_coefficients, None, new_camera_matrix)

In [27]:
# Plot the undistorted image points on the undistorted image
undistorted_img = cv2.cvtColor(undistorted_img, cv2.COLOR_BGR2RGB)
plt.figure(figsize=(20,20),dpi=100)
plt.scatter(undistorted_image_points[:,0,0], undistorted_image_points[:,0,1], c='r', s=100)
plt.imshow(undistorted_img)
plt.show()


In [28]:
# Save the pixel coordinates of the undistorted image points
np.savetxt(f'{imgpath[:-4]}_undistorted_image_points_2D.txt', undistorted_image_points[:,0,:])

In [29]:
def create_projection_matrix(rotation_vectors, translation_vectors, camera_matrix):
    """
    Create the 3x4 projection matrix
    :param rotation_vectors: rotation vectors
    :param translation_vectors: translation vectors
    :param camera_matrix: camera matrix
    """
    R = cv2.Rodrigues(rotation_vectors[0])[0]
    t = translation_vectors[0]
    Rt = np.concatenate([R,t], axis=-1) # [R|t]
    P = np.matmul(camera_matrix,Rt) # A[R|t]
    return P

P = create_projection_matrix(rotation_vectors, translation_vectors, new_camera_matrix)

# Visualize the 3D points using the projection matrix
homogeneous_world_points = np.concatenate([world_points, np.ones((world_points.shape[0],1))], axis=-1)
homogeneous_image_points = np.matmul(P, homogeneous_world_points.T).T
image_points = homogeneous_image_points[:,0:2] / homogeneous_image_points[:,2:3]
plt.figure(figsize=(20,20),dpi=100)
plt.scatter(image_points[:,0], image_points[:,1], c='r', s=10)
plt.imshow(undistorted_img)
plt.show()

# Save the 3x4 projection matrix

np.savetxt(f'{imgpath[:-4]}_projection_matrix.txt', P)